## 处理疾病诊断分类
* 疾病实体含有的可能属性并集
* 疾病实体
* 科室实体
* 症状实体
* 发病部位
* 多发群体

In [1]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

#Getting a Database
db = client['baidubaike']

In [2]:
c = db['symptomsdetail']

In [94]:
#提取所有疾病实体的属性
s = set()
for t in c.find(no_cursor_timeout=True):
#     print(type(t))
#     print(set(t.keys()))
    s = s.union(set(t.keys()))
#     break

len(s)#826

#list(s)

826

In [113]:
#提取所有疾病实体
diseases = set()
for t in c.find(no_cursor_timeout=True):
    diseases.add(t['title'])
len(diseases)#6771，和爬取网页数相同，证明没有重复

6771

In [96]:
#提取就诊科室
department = set()
for t in c.find(no_cursor_timeout=True):
    if 'dd就诊科室' in list(t.keys()):  
        t['dd就诊科室'] = t['dd就诊科室'].replace('、','，').replace('：','，').replace('；','，').replace('。','，').replace('或','，').replace('和','，').replace('等','').replace('"','')
        tt = t['dd就诊科室'].split('，')
        for i in range(len(tt)):
            if tt[i] == '外壳':
                tt[i] = '外科'
            if not tt[i].endswith('科'):
                tt[i] = tt[i] + '科'
            
        department = department.union(list(map(lambda x:x.strip(),tt)))
                         
len(department)#156

156

In [97]:
#提取症状
symptom = set()
for t in c.find(no_cursor_timeout=True):
    if 'dd常见症状' in list(t.keys()):  
        t['dd常见症状'] = t['dd常见症状'].replace('、','，').replace('：','，').replace('；','，').replace('。','，').replace('或','，').replace('和','，').replace('等','').replace('"','')
        tt = t['dd常见症状'].split('，')
        symptom = symptom.union([i for i in list(map(lambda x:x.strip(),tt)) if i!='' and 1<len(i)<11])
                         
len(symptom)#12362

12362

In [98]:
#提取发病部位
position = set()
for t in c.find(no_cursor_timeout=True):
    if 'dd常见发病部位' in list(t.keys()):  
        t['dd常见发病部位'] = t['dd常见发病部位'].replace('、','，').replace('：','，').replace('；','，').replace('。','，').replace('或','，').replace('和','，').replace('等','').replace('"','')
        tt = t['dd常见发病部位'].split('，')
        position = position.union([i for i in list(map(lambda x:x.strip(),tt)) if i!='' and 1<len(i)<11])
                         
len(position)#1405

1405

In [99]:
#提取多发群体
group = set()
for t in c.find(no_cursor_timeout=True):
    if 'dd多发群体' in list(t.keys()):  
        t['dd多发群体'] = t['dd多发群体'].replace('、','，').replace('：','，').replace('；','，').replace('。','，').replace('或','，').replace('和','，').replace('等','').replace('"','')
        tt = t['dd多发群体'].split('，')
        group = group.union([i for i in list(map(lambda x:x.strip(),tt)) if i!='' and 1<len(i)<11])

group = group - department                         
len(group)#1476

1476

In [100]:
position = position - department

In [101]:
position = position - set(['十二指肠溃疡','海绵状血管瘤','皮肤平滑肌瘤','角膜炎'])

In [114]:
diseases = diseases - set(['胸锁乳突肌','胎盘'])

In [103]:
l = ['自身免疫性疾病',
'糖尿病',
'心脏病',
'高钙血症',
'乳腺癌',
'前列腺癌',
'四环素牙',
'支气管哮喘',
'糖尿病性视网膜病变',
'高血压',
'毛发红糠疹',
'心肌炎',
'肾癌',
'硬皮病',
'动脉硬化',
'肺气肿',
'高脂血症',
'皮肌炎',
'意识障碍',
'心绞痛',
'脑卒中',
'干燥综合征',
'血脂异常',
'肝硬化',
'肺癌',
'先天性心脏病',
'银屑病',
'高脂蛋白血症',
'关节炎',
'白血病',
'妊娠期高血压疾病',
'氟斑牙',
'包皮过长',
'肿瘤',
'甲状腺癌',
'系统性红斑狼疮',
'静脉曲张']
group = group - set(l)

In [115]:
l = ['胎儿','足月小样儿','早产儿','过期产儿','女性生殖系统']
diseases = diseases - set(l)

In [105]:
l = ['肺脏',
'眼角膜',
'筋骨',
'头皮',
'视网膜',
'躯干',
'眼球',
'晶状体',
'阴茎',
'胸骨',
'肌肉',
'脐部',
'眼睛',
'关节'
]
group = group - set(l)

In [106]:
l = ['中年以上男性',
'儿童',
'孕妇',
'成人',
'艾滋病患者',
'婴儿期',
'青壮年男性',
'产妇',
'1～6岁儿童',
'小儿']
position = position - set(l)

In [107]:
l = position & group
position = position - set(l)
group = group - set(l)
position & group

set()

In [108]:
symptom = symptom - {'儿童', '婴幼儿', '新生儿', '早产儿', '饮酒',}
group = group - {'冠心病', '多发性硬化', '忧虑', '癫痫', '高血脂', '高血钾',}

In [109]:
symptom = symptom - {'先天性', '原发性', '心跳', '急性', '焦急'}
group = group - {'先天性', '原发性', '心跳', '急性', '焦急'}
symptom & group

set()

In [110]:
l = symptom & position
symptom = symptom - set(l)

In [117]:
symptom = symptom - (diseases & symptom)

In [118]:
import itertools
item = ['diseases','symptom','position','department','group']
cc = list(itertools.combinations(item,2))
count = 0
for a,b in itertools.combinations([diseases,symptom,position,department,group],2):
    print(cc[count])
    print(len(a & b))
    count += 1

('diseases', 'symptom')
0
('diseases', 'position')
0
('diseases', 'department')
0
('diseases', 'group')
0
('symptom', 'position')
0
('symptom', 'department')
0
('symptom', 'group')
0
('position', 'department')
0
('position', 'group')
0
('department', 'group')
0


In [120]:
item = ['disease','symptom','position','department','group']
count = 0
for i in [diseases,symptom,position,department,group]:
    print(item[count],len(i))
    count += 1

disease 6764
symptom 11745
position 1384
department 156
group 1409


In [121]:
import pickle
with open('diseaseattr.pkl', 'wb') as f:
    pickle.dump(s, f)
    
with open('diseases.pkl', 'wb') as f:
    pickle.dump(diseases, f)
    
with open('symptoms.pkl', 'wb') as f:
    pickle.dump(symptom, f)
    
with open('positions.pkl', 'wb') as f:
    pickle.dump(position, f)
    
with open('departments.pkl', 'wb') as f:
    pickle.dump(department, f)
    
with open('groups.pkl', 'wb') as f:
    pickle.dump(group, f)